In [1]:
import pandas as pd
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import time
from IPython.display import clear_output
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

In [35]:
#Preprocess training set
data=pd.read_csv('C:/Users/X99S5/Documents/AMLS_22-23_SN18046828/Datasets/cartoon_set/labels.csv')

labelsTrain = data["\teye_color\tface_shape\tfile_name"].str.split(pat="\t", n=-1, expand=True)
labelsTrain.drop(columns =[0,2], inplace = True)
labelsTrain.columns = [ "eye_color" , "img_name"]
labelsTrain = labelsTrain.astype({'eye_color': 'int32'})
labelsTrain = labelsTrain.astype({'img_name': 'string'})

#Preprocess testing set
data=pd.read_csv('C:/Users/X99S5/Documents/AMLS_22-23_SN18046828/Datasets/cartoon_set_test/labels.csv')

labelsTest = data["\teye_color\tface_shape\tfile_name"].str.split(pat="\t", n=-1, expand=True)
labelsTest.drop(columns =[0,2], inplace = True)
labelsTest.columns = [ "eye_color" , "img_name"]
labelsTest = labelsTest.astype({'eye_color': 'int32'})
labelsTest = labelsTest.astype({'img_name': 'string'})

In [3]:
#230:270,180:320

In [4]:
#Load training set################################
imagesTrain = np.zeros((10000, 50 , 50, 3)) 

for i in range(0,10000):
    image = cv.imread('C:/Users/X99S5/Documents/AMLS_22-23_SN18046828/Datasets/cartoon_set/img/' + labelsTrain["img_name"][i])
    image = image[230:290,180:320] # cut 95 from left / right sides , 75 from top, 80 from the bottom
    image = cv.resize(image, dsize=(50 , 50), interpolation=cv.INTER_CUBIC)
    imagesTrain[i] = image
    
imagesTrain = imagesTrain.reshape(10000,7500)    
imagesTrain = pd.DataFrame(imagesTrain) 


#Load Testing set################################
imagesTest = np.zeros((2500, 50 , 50, 3))

for i in range(0,2500):
    image = cv.imread('C:/Users/X99S5/Documents/AMLS_22-23_SN18046828/Datasets/cartoon_set_test/img/' + labelsTest["img_name"][i])
    image = image[230:290,180:320]
    image = cv.resize(image, dsize=(50 , 50), interpolation=cv.INTER_CUBIC)
    imagesTest[i] = image
    
    
imagesTest = imagesTest.reshape(2500,7500)   
imagesTest = pd.DataFrame(imagesTest) 

,0,1,2,3,4,5,6,7,8,9,...,7490,7491,7492,7493,7494,7495,7496,7497,7498,7499
0,34.0,106.0,148.0,34.0,106.0,148.0,64.0,128.0,173.0,123.0,...,239.0,75.0,80.0,239.0,73.0,78.0,235.0,20.0,23.0,59.0
1,119.0,113.0,99.0,119.0,113.0,99.0,117.0,111.0,98.0,102.0,...,117.0,52.0,54.0,60.0,17.0,20.0,29.0,48.0,61.0,95.0
2,110.0,161.0,217.0,110.0,161.0,217.0,110.0,161.0,217.0,110.0,...,239.0,75.0,80.0,239.0,70.0,75.0,225.0,28.0,31.0,95.0
3,121.0,140.0,169.0,19.0,35.0,62.0,19.0,35.0,62.0,20.0,...,246.0,193.0,216.0,246.0,193.0,216.0,246.0,193.0,216.0,246.0
4,120.0,114.0,101.0,119.0,113.0,99.0,119.0,113.0,99.0,120.0,...,246.0,193.0,216.0,246.0,193.0,216.0,246.0,193.0,216.0,246.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,126.0,180.0,227.0,113.0,162.0,214.0,52.0,83.0,155.0,29.0,...,227.0,126.0,180.0,227.0,126.0,180.0,227.0,126.0,180.0,227.0
9996,96.0,153.0,178.0,112.0,165.0,196.0,146.0,192.0,240.0,146.0,...,240.0,146.0,192.0,240.0,146.0,192.0,240.0,146.0,192.0,240.0
9997,187.0,212.0,242.0,193.0,216.0,246.0,193.0,216.0,246.0,193.0,...,253.0,195.0,218.0,246.0,193.0,216.0,246.0,185.0,210.0,239.0
9998,126.0,180.0,227.0,126.0,180.0,227.0,126.0,180.0,227.0,126.0,...,12.0,43.0,37.0,40.0,105.0,109.0,118.0,106.0,114.0,124.0


In [9]:
x_train=np.zeros((10000,7502))
for i in range(0,10000):
    x_train[i]=np.convolve(imagesTrain.iloc[i,:],[0.066,-5.599,5.566])
    
x_test=np.zeros((2500,7502))
for i in range(0,2500):
    x_test[i]=np.convolve(imagesTest.iloc[i,:],[0.066,-5.599,5.566])

x_train = pd.DataFrame(x_train) 
x_test = pd.DataFrame(x_test)


In [14]:
for i in range(100,1000,100):
    model = KNeighborsClassifier(n_neighbors=i, weights = 'distance' , n_jobs=-1)
    model.fit(x_train,labelsTrain['eye_color'])
    print(model.score(x_test,labelsTest['eye_color']))

0.8152
0.8272
0.8292
0.8308
0.8296
0.8324
0.8312
0.8344
0.8304


In [19]:

hyper_params = {
    'weights' : ['uniform', 'distance'],
    'n_neighbors' : [100, 300, 500, 800, 1000],
    'p' : [1 , 2],

}

In [20]:
model = KNeighborsClassifier(n_jobs = -1)
grid=GridSearchCV(model , param_grid=hyper_params , cv=10 , n_jobs=-1 ,  verbose=10)

In [21]:
grid.fit(x_train,labelsTrain['eye_color'])

Fitting 10 folds for each of 20 candidates, totalling 200 fits


GridSearchCV(cv=10, estimator=KNeighborsClassifier(n_jobs=-1), n_jobs=-1,
             param_grid={'n_neighbors': [100, 300, 500, 800, 1000], 'p': [1, 2],
                         'weights': ['uniform', 'distance']},
             verbose=10)

In [22]:
grid.best_estimator_


KNeighborsClassifier(n_jobs=-1, n_neighbors=100, p=1, weights='distance')

In [23]:
grid.best_score_

0.8322999999999998

In [24]:
model = KNeighborsClassifier(n_jobs=-1, n_neighbors=100, p=1, weights='distance')
model.fit(x_train,labelsTrain['eye_color'])
print(model.score(x_test,labelsTest['eye_color']))

0.8236


In [26]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten
print("num of gpus available: ",len(tf.config.experimental.list_physical_devices('GPU')))
from keras.utils import normalize,to_categorical
from keras.preprocessing.image import ImageDataGenerator
tf.config.run_functions_eagerly(True)

num of gpus available:  1


In [36]:
#Load training set for CNN################################
imagesTrain = np.zeros((10000, 150 , 150, 3)) 

for i in range(0,10000):
    image = cv.imread('C:/Users/X99S5/Documents/AMLS_22-23_SN18046828/Datasets/cartoon_set/img/' + labelsTrain["img_name"][i])
    image = image[75:420,95:405] # cut 95 from left / right sides , 75 from top, 80 from the bottom
    image = cv.resize(image, dsize=(150 , 150), interpolation=cv.INTER_CUBIC)
    # imagesTrain[i] = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    imagesTrain[i] = image
#imagesTrain = imagesTrain.reshape(10000,26815)    
#imagesTrain = pd.DataFrame(imagesTrain) 


#Load Testing set################################
imagesTest = np.zeros((2500, 150 , 150, 3))

for i in range(0,2500):
    image = cv.imread('C:/Users/X99S5/Documents/AMLS_22-23_SN18046828/Datasets/cartoon_set_test/img/' + labelsTest["img_name"][i])
    image = image[75:420,95:405]
    image = cv.resize(image, dsize=(150 , 150), interpolation=cv.INTER_CUBIC)
    #imagesTest[i] = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    imagesTest[i] = image
    
#imagesTest = imagesTest.reshape(2500,26815)   
#imagesTest = pd.DataFrame(imagesTest) 

In [37]:
labelsTrain = labelsTrain['eye_color']
labelsTest = labelsTest['eye_color']

imagesTrain = normalize(imagesTrain,axis=1)
imagesTest = normalize(imagesTest,axis=1)

labelsTrain = to_categorical(labelsTrain)
labelsTest = to_categorical(labelsTest)

In [38]:
datagen_train = ImageDataGenerator()
#needed to save memory

datagen_Test = ImageDataGenerator()

In [39]:
model = Sequential()

model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(150,150,3)))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Flatten())

model.add( Dense(128,activation='relu') )
model.add( Dense(64,activation='relu') )
model.add( Dense(5,activation='softmax') )


In [40]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 74, 74, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 17, 17, 128)     

In [41]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history = model.fit( 
    datagen_train.flow(imagesTrain, labelsTrain, batch_size=100),
    epochs=10,
    validation_data=datagen_Test.flow(imagesTest, labelsTest, batch_size=100),
)

C:\Users\X99S5\AppData\Roaming\Python\Python310\site-packages\tensorflow\python\data\ops\structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/10
100/100 [==============================] - 23s 116ms/step - loss: 1.3901 - accuracy: 0.3798 - val_loss: 1.1323 - val_accuracy: 0.4440
Epoch 2/10
100/100 [==============================] - 12s 116ms/step - loss: 0.8379 - accuracy: 0.6192 - val_loss: 0.6004 - val_accuracy: 0.7052
Epoch 3/10
100/100 [==============================] - 12s 116ms/step - loss: 0.5609 - accuracy: 0.7575 - val_loss: 0.4092 - val_accuracy: 0.8244
Epoch 4/10
100/100 [==============================] - 12s 118ms/step - loss: 0.3992 - accuracy: 0.8220 - val_loss: 0.3590 - val_accuracy: 0.8300
Epoch 5/10
100/100 [==============================] - 12s 116ms/step - loss: 0.3486 - accuracy: 0.8433 - val_loss: 0.3572 - val_accuracy: 0.8360
Epoch 6/10
100/100 [==============================] - 12s 116ms/step - loss: 0.3352 - accuracy: 0.8484 - val_loss: 0.3467 - val_accuracy: 0.8352
Epoch 7/10
100/100 [==============================] - 12s 117ms/step - loss: 0.3186 - accuracy: 0.8582 - val_loss: 0.3353 - val_ac